In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydicom
from PIL import Image, ImageEnhance


## CT Abnormality Dataset

In [2]:
df_ab = pd.read_csv('data/nlst_567/nlst_567_ct_ab_20191108.csv')
df_ab.head()

,sct_ab_desc,sct_ab_num,sct_epi_loc,sct_long_dia,sct_margins,sct_perp_dia,sct_pre_att,study_yr,sct_slice_num,sct_found_after_comp,pid,dataset_version,scr_group,ct_selected,sct_ab_preexist,sct_ab_attn,sct_ab_gwth,visible_days,sct_ab_has_comparison
0,51,1,1.0,8.0,2.0,5.0,1.0,0,38.0,0.0,100012,2011.02.03/08.20.19,1,1,NaN,NaN,NaN,NaN,0
1,64,2,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,100012,2011.02.03/08.20.19,1,1,NaN,NaN,NaN,NaN,0
2,51,1,1.0,15.0,1.0,10.0,2.0,1,39.0,0.0,100012,2011.02.03/08.20.19,1,1,2.0,2.0,2.0,16.0,1
3,64,2,NaN,NaN,NaN,NaN,NaN,1,NaN,0.0,100012,2011.02.03/08.20.19,1,1,NaN,NaN,NaN,NaN,0
4,65,1,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,100147,2011.02.03/08.20.19,1,1,NaN,NaN,NaN,NaN,0


In [3]:
# drop entries w/o corresponding slice number
df_ab = df_ab.dropna(subset=['sct_slice_num'])

# extract only pid, study year and slice number
df_slices = df_ab[['pid', 'study_yr', 'sct_ab_num', 'sct_slice_num', 'sct_long_dia']]
print(df_slices.shape)
df_slices.head()

# convert years to match CT image file structure 
year_conversion = {0: 'T0', 1: 'T1', 2: 'T2'}
df_slices['study_yr'] = df_slices.apply(lambda row: year_conversion[row['study_yr']], axis=1)
df_slices.head()

(1706, 5)


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,pid,study_yr,sct_ab_num,sct_slice_num,sct_long_dia
0,100012,T0,1,38.0,8.0
2,100012,T1,1,39.0,15.0
7,100147,T0,4,88.0,9.0
8,100147,T1,1,92.0,NaN
15,100158,T2,1,57.0,12.0


# Treatment dataset

In [4]:
df_treat = pd.read_csv('data/nlst_567/treatment.data.d100517.csv')
df_treat.head()

,dataset_version,pid,disease_post_surg,rad_stop_days,treat,treat_days,treat_year,treatnum
0,2011.02.03/10.05.17,100012,0.0,NaN,2,483.0,1,203
1,2011.02.03/10.05.17,100012,0.0,NaN,2,483.0,1,208
2,2011.02.03/10.05.17,100049,NaN,2499.0,1,2471.0,6,101
3,2011.02.03/10.05.17,100055,2.0,NaN,2,148.0,0,201
4,2011.02.03/10.05.17,100055,2.0,NaN,2,148.0,0,208


In [5]:
# keep only patients with no residual disease left after surgery 
df_treat = df_treat[df_treat.disease_post_surg == 0]

# keep only pids 
df_treat = df_treat[['pid', 'treat_year', 'treat_days', 'disease_post_surg']]
df_treat['pid'] = df_treat['pid'].drop_duplicates()
df_treat = df_treat.dropna()
df_treat.head()

,pid,treat_year,treat_days,disease_post_surg
0,100012.0,1,483.0,0.0
6,100147.0,1,491.0,0.0
8,100158.0,2,795.0,0.0
11,100242.0,0,79.0,0.0
15,100280.0,1,456.0,0.0


# Person dataset

In [6]:
df_person = pd.read_csv('data/nlst_567/nlst_567_prsn_20191108.csv')

# only keep patients that remained in contact
df_person = df_person[np.logical_or(df_person['contactstatus'] == 1, df_person['contactstatus'] == 2)]

# combine death to lung cancer columns 
df_person['finaldeathlc'] =  df_person['finaldeathlc'].fillna(0)
df_person['dcfdeathlc'] =  df_person['dcfdeathlc'].fillna(0)
df_person['finaldeathlc'] = np.logical_or(df_person['finaldeathlc'], df_person['dcfdeathlc'])

# keep only pid, progression status and death columns
df_person = df_person[['pid', 'prog_days_1st', 'progressed_ever', 'progsite_adrenal_ever', 'finaldeathlc', 'death_days', 'last_progfree_days']]


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (201,224,225) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Join datasets

In [7]:
df = df_slices.join(df_treat.set_index('pid'), on='pid', how='inner')
df = df.join(df_person.set_index('pid'), on='pid', how='inner')
# remove uncertains 
df = df[np.logical_or(
    np.logical_or(df.progressed_ever == 0, df.progressed_ever == 1),
    np.logical_or(df.progsite_adrenal_ever == 0, df.progsite_adrenal_ever == 1)
)]

# Clean

In [8]:
# drop those who had no progression but died of lung cancer (likely died in surgery?)
df = df.drop(df[np.logical_and(df.progressed_ever == 0, df.finaldeathlc == 1)].index)

In [9]:
# drop those w/o diameter information (< 4mm)
df = df.dropna(subset=['sct_long_dia'])

In [10]:
# make recurrence column
df_slices_pos = df[np.logical_or(
    np.logical_and(df.progressed_ever == 1, df.prog_days_1st >  df.treat_days),
    df.finaldeathlc == 1
)]
df_slices_neg = df[np.logical_and(
    np.logical_or(df.progressed_ever == 0, df.prog_days_1st <  df.treat_days), 
    df.finaldeathlc == 0
)]

recurrence = []
for idx in df.index:
    if idx in df_slices_pos.index: 
        recurrence.append(1)
    elif idx in df_slices_neg.index:
        recurrence.append(0)
    else:
        recurrence.append(None)
df['recurrence'] = recurrence

In [11]:
df_slices_pos = df[df.recurrence == 1]
df_slices_neg = df[df.recurrence == 0]

print(f"Number of 'recurrent' nodules: {len(df_slices_pos)}")
print(f"Number of 'recurrent' patients: {len(df_slices_pos.pid.unique())}\n")

print(f"Number of 'non-recurrent' nodules: {len(df_slices_neg)}")
print(f"Number of 'non-recurrent' patients: {len(df_slices_neg.pid.unique())}")

Number of 'recurrent' nodules: 233
Number of 'recurrent' patients: 99

Number of 'non-recurrent' nodules: 875
Number of 'non-recurrent' patients: 299


In [12]:
# save table
df.to_csv('data/nlst_table.csv', index=None)

In [13]:
df.columns

Index(['pid', 'study_yr', 'sct_ab_num', 'sct_slice_num', 'sct_long_dia',
       'treat_year', 'treat_days', 'disease_post_surg', 'prog_days_1st',
       'progressed_ever', 'progsite_adrenal_ever', 'finaldeathlc',
       'death_days', 'last_progfree_days', 'recurrence'],
      dtype='object')

# More Info

**Infer adjuvent therapy feature**

In [14]:
df_patient = df[['pid', 'treat_days', 'recurrence']]
df_patient = df_patient.drop_duplicates()

df_chemo = pd.read_csv('data/nlst_567/treatment.data.d100517.csv')
df_chemo = df_chemo[df_chemo.treatnum==300]
ChemoDays = {row.pid: row.treat_days for _, row in df_chemo.iterrows()}

adjuvent = []
for idx, row in df_patient.iterrows():
    if row.pid in ChemoDays and row.treat_days < ChemoDays[row.pid]:
        adjuvent.append(1)
    else:
        adjuvent.append(0)
        
        
df_patient['adjuvent'] = adjuvent

more_feats = pd.read_csv('data/nlst_567/nlst_567_prsn_20191108.csv')[['pid', 'age', 'gender', 'de_stag']]
df_patient = df_patient.join(more_feats.set_index('pid'), on='pid', how='inner')

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (201,224,225) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


**Rearrange staging data**

In [15]:
df_patient['stage 1a'] = [int(de_stag==110) for de_stag in df_patient.de_stag]
df_patient['stage 1b'] = [int(de_stag==120) for de_stag in df_patient.de_stag]
df_patient['stage 2a'] = [int(de_stag==210) for de_stag in df_patient.de_stag]
df_patient['stage 2b'] = [int(de_stag==220) for de_stag in df_patient.de_stag]
df_patient['stage 3a'] = [int(de_stag==310) for de_stag in df_patient.de_stag]
df_patient['stage 3b'] = [int(de_stag==320) for de_stag in df_patient.de_stag]
df_patient['stage 4'] = [int(de_stag==400) for de_stag in df_patient.de_stag]
df_patient = df_patient.drop(['de_stag', 'treat_days'], axis=1)
df_patient = df_patient.dropna()

df_patient['stage 1'] = np.logical_or(df_patient['stage 1a'], df_patient['stage 1b'])
df_patient['stage 2'] = np.logical_or(df_patient['stage 2a'], df_patient['stage 2b'])
df_patient['stage 3'] = np.logical_or(df_patient['stage 3a'], df_patient['stage 3b'])

df_patient['stage'] = df_patient['stage 1'] + 2*df_patient['stage 2'] + 3*df_patient['stage 3'] + 4*df_patient['stage 4']

In [16]:
df_patient.to_csv('data/nlst_patient_table.csv', index=None)